In [ ]:
# ===========
# 1. Enable data aug with tfms_side_on, etc and precompute = True
# 2. Use lr_find() to find the highest learning rate
# 3. Train last layer
# 4. Train last layer with data aug
# 5. Unfreeze layers with learn.unfreeze()
# 6. Set layers to 3-10 times lower learning rate for earlier layers with np array
#     1. Ex: lr=np.array([1e-4,1e-3,1e-2])
# 7. Use lr_find() again
# 8. Train network with cycle_mult = 2 until it overfits
# ===========

%reload ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

torch.cuda.set_device(1)

PATH = "data/dogbreed"
sz = 224 #image size
arch = resnext101_64 #use resnext50 for better results
bs = 58 #batch size

label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n) #returns around 20% of rows for a validation set since we dont have one

!ls {PATH} #It turns out that the train file is not sorted into dogs, cats, etc. Instead we have a csv with all the types
label_df = pd.read_csv(label_csv) #Pandas function for reading csvs
label_df.head() #Gets the first row of csv

# ===========
# This df.head returns this sort of table:
# {row_num} {ID} {Breed}
# ===========

label_df.pivot_table(index = 'breed', aggfunc = len).sort_values('id', ascending = False) #Sorts this into how many of each breed (this is 	why we have aggfunc)

# ~~~~~~~~~~~~
# We then find out that there are 120 breeds since there are 120 rows
# ~~~~~~~~~~~~

tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1) #Pictures of dogs so we do side_on. Max_zoom is  	for data augs
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv, test_name = 'test', val_idxs = val_idxs, suffix = '.jpg', tfms = tfms, 	bs = bs)
fn = PATH+data.trn_ds.fnames[0]; fn
img = PIL.image.open(fn); img
img.size

size_d = {k: PIL.image.open(PATH+k).size for k in data.trn_ds.fnames} #Dict maps name to filenames array
row_sz, col_sz = list(zip(*size_d.values()))
row_sz = np.array(row_sz); col_sz = np.array(col_sz)
row_sz[:5] #First 5 rows
plt.hist(row_sz);
plt.hist(row_sz[row_sz<1000]) #np slicing to get row sizes less than 1000
plt.hist(col_sz);
len(data.trn_ds), len(data.test_ds)
len(data.classes), data.classes[:5]

def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1) #Pictures of dogs so we do side_on. Max_zoom 		is  for data augs
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv, test_name = 'test', val_idxs = val_idxs, suffix = '.jpg', tfms = 		tfms, bs = bs)
    return data if sz>300 else data.resize(340, 'tmp')

# ===========
# PRECOMPUTE
# ===========

data = get_data(sz, bs)
learn = ConvLearner.pretrained(arch, data, precompute = True) #sets up model
lrf = learn.lr_find()
learn.fit(1e-2, 5) #does the learning

learn.precompute = False
learn.fit(1e-2, 5, cycle_len = 1)
learn.save('224_pre')
learn.load('224_pre')

learn.unfreeze()
lr = np.array([1e-4, 1e-3, 1e-2]) #Differential learning rates
learn.fit(lr, 3, cycle_len = 1)

learn.set_data(get_data(299, bs)) #increase image size
learn.freeze()

learn.fit(1e-2, 3, cycle_len = 1, cycle_mult = 2) #runs first cycle once, second twice, third 4 times, etc. This increases validation accuracy
log_preds, y= learn.TTA() #assigns first element to log_preds and second to y
probs = np.exp(log_preds)
accuracy(log_preds, y), metrics.log_loss(y, probs)
